<a href="https://colab.research.google.com/github/sheikh495/ComputerSystemsOrganization/blob/main/COSC_530_ProgrammingAssignment)2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
class Instruction:
    def __init__(self, opcode, operands, address):
        self.opcode = opcode
        self.operands = operands
        self.address = address
        self.issue_cycle = None
        self.execute_start_cycle = None
        self.execute_end_cycle = None
        self.read_result_cycle = None
        self.commit_cycle = None

class Processor:
    def __init__(self, config_file):
        self.buffer_sizes = {}
        self.latencies = {}
        self.instructions = []
        self.reorder_buffer = []
        self.memory = {}
        self.load_buffer = []
        self.cycle = 1
        self.read_config(config_file)

    def read_config(self, config_file):
        with open(config_file, 'r') as file:
            config_lines = file.readlines()

        section = None
        for line in config_lines:
            line = line.strip()
            if line == "buffers":
                section = "buffers"
            elif line == "latencies":
                section = "latencies"
            elif section == "buffers" and line:
                name, value = line.split(':')
                self.buffer_sizes[name.strip()] = int(value)
            elif section == "latencies" and line:
                name, value = line.split(':')
                self.latencies[name.strip()] = int(value)

    def read_trace(self, trace_file):
        with open(trace_file, 'r') as file:
            trace_lines = file.readlines()

        for line in trace_lines:
            opcode, operands = line.strip().split(' ', 1)
            if opcode in ['flw', 'lw', 'fsw', 'sw']:
                instr, address = operands.split(':')
                address = int(address)
            else:
                instr = operands
                address = None

            instruction = Instruction(opcode, instr, address)
            self.instructions.append(instruction)

    def execute_instruction(self, instruction):
        if instruction.opcode in ['flw', 'lw', 'fsw', 'sw']:
            # Memory access operation
            if instruction.opcode in ['flw', 'lw']:
                self.load_buffer.append(instruction)
            elif instruction.opcode in ['fsw', 'sw']:
                self.memory[instruction.address] = instruction.operands
        else:
            # Integer or floating-point operation
            latency = self.latencies.get(f"{instruction.opcode}_add", 1)
            instruction.execute_start_cycle = self.cycle
            instruction.execute_end_cycle = self.cycle + latency

    def issue_instruction(self, instruction):
        if len(self.reorder_buffer) < self.buffer_sizes.get("reorder", 5):
            self.reorder_buffer.append(instruction)
            instruction.issue_cycle = self.cycle

    def commit_instruction(self, instruction):
        self.reorder_buffer.remove(instruction)
        instruction.commit_cycle = self.cycle

    def process_instructions(self):
        for instruction in self.instructions:
            self.issue_instruction(instruction)

            if instruction.opcode in ['flw', 'lw', 'fsw', 'sw']:
                while len(self.load_buffer) > 0 and self.load_buffer[0].commit_cycle:
                    self.load_buffer.pop(0)
            else:
                while len(self.reorder_buffer) > 0 and self.reorder_buffer[0].commit_cycle:
                    self.commit_instruction(self.reorder_buffer[0])

            self.execute_instruction(instruction)

            # Set read_result_cycle only when the instruction is executed
            if instruction.execute_end_cycle:
                instruction.read_result_cycle = instruction.execute_end_cycle + 1

            self.commit_instruction(instruction)
            self.cycle = instruction.read_result_cycle if instruction.read_result_cycle else self.cycle

    def print_simulation_results(self):
        print("Pipeline Simulation")
        print("-" * 60)
        print("                          Memory Writes")
        print("Instruction          Issues  Executes  Read    Result  Commits")
        print("                    -------  -------- ------  ------  -------")

        for instruction in self.instructions:
            if instruction.execute_start_cycle:
                exec_range = f"{instruction.execute_start_cycle}-{instruction.execute_end_cycle}"
            else:
                exec_range = "-"

            if instruction.commit_cycle:
                commit_cycle = str(instruction.commit_cycle)
            else:
                commit_cycle = "-"

            print(f"{instruction.opcode} {instruction.operands}{' ' * (15 - len(instruction.operands))} {instruction.issue_cycle}{' ' * (8 - len(str(instruction.issue_cycle)))} "
                  f"{exec_range}{' ' * (10 - len(exec_range))} "
                  f"{instruction.read_result_cycle}{' ' * (8 - len(str(instruction.read_result_cycle)))} "
                  f"{commit_cycle}")

    def print_delays(self):
        print("\nDelays")
        print("-" * 30)
        print(f"reorder buffer delays: {sum(1 for instr in self.instructions if instr.issue_cycle and not instr.commit_cycle)}")
        print(f"reservation station delays: {sum(1 for instr in self.instructions if instr.issue_cycle and not instr.execute_start_cycle)}")
        print(f"data memory conflict delays: {sum(1 for instr in self.instructions if instr.opcode in ['flw', 'lw', 'fsw', 'sw'] and instr.execute_start_cycle and not instr.read_result_cycle)}")
        print(f"true dependence delays: {sum(1 for instr in self.instructions if instr.opcode not in ['flw', 'lw', 'fsw', 'sw'] and instr.execute_start_cycle and not instr.read_result_cycle)}")


# Example usage:
if __name__ == "__main__":
    config_file = "/content/config.txt"
    trace_file = "/content/trace.dat"

    processor = Processor(config_file)
    processor.read_trace(trace_file)
    processor.process_instructions()

    print("Configuration")
    print("-------------")
    # Read and print the pipeline configuration from config_file
    with open(config_file, 'r') as file:
        config = file.read()
        print(config)

    processor.print_simulation_results()
    processor.print_delays()


Configuration
-------------
buffers

eff addr: 2
fp adds: 3
fp muls: 3
ints: 2
reorder: 5

latencies

fp_add: 2
fp_sub: 2
fp_mul: 5
fp_div: 10


Pipeline Simulation
------------------------------------------------------------
                          Memory Writes
Instruction          Issues  Executes  Read    Result  Commits
                    -------  -------- ------  ------  -------
flw    f6,32(x2)    1        -          None     1
flw    f2,48(x3)    1        -          None     1
fmul.s f0,f2,f4        1        1-2        3        1
fsub.s f8,f6,f2        3        3-4        5        3
fdiv.s f10,f0,f6       5        5-6        7        5
fadd.s f6,f8,f2        7        7-8        9        7

Delays
------------------------------
reorder buffer delays: 0
reservation station delays: 2
data memory conflict delays: 0
true dependence delays: 0


#Final

In [29]:
class Instruction:
    def __init__(self, opcode, operands, address):
        self.opcode = opcode
        self.operands = operands
        self.address = address
        self.issue_cycle = None
        self.execute_start_cycle = None
        self.execute_end_cycle = None
        self.read_result_cycle = None
        self.commit_cycle = None

class Processor:
    def __init__(self, config_file):
        self.buffer_sizes = {}
        self.latencies = {}
        self.instructions = []
        self.reorder_buffer = []
        self.memory = {}
        self.load_buffer = []
        self.cycle = 1
        self.read_config(config_file)

    def read_config(self, config_file):
        with open(config_file, 'r') as file:
            config_lines = file.readlines()

        section = None
        for line in config_lines:
            line = line.strip()
            if line == "buffers":
                section = "buffers"
            elif line == "latencies":
                section = "latencies"
            elif section == "buffers" and line:
                name, value = line.split(':')
                self.buffer_sizes[name.strip()] = int(value)
            elif section == "latencies" and line:
                name, value = line.split(':')
                self.latencies[name.strip()] = int(value)

    def read_trace(self, trace_file):
        with open(trace_file, 'r') as file:
            trace_lines = file.readlines()

        for line in trace_lines:
            opcode, operands = line.strip().split(' ', 1)
            if opcode in ['flw', 'lw', 'fsw', 'sw']:
                instr, address = operands.split(':')
                address = int(address)
            else:
                instr = operands
                address = None

            instruction = Instruction(opcode, instr, address)
            self.instructions.append(instruction)

    def execute_instruction(self, instruction):
        if instruction.opcode in ['flw', 'lw', 'fsw', 'sw']:
            # Memory access operation
            if instruction.opcode in ['flw', 'lw']:
                self.load_buffer.append(instruction)
            elif instruction.opcode in ['fsw', 'sw']:
                self.memory[instruction.address] = instruction.operands
        else:
            # Integer or floating-point operation
            latency = self.latencies.get(f"{instruction.opcode}_add", 1)
            instruction.execute_start_cycle = self.cycle
            instruction.execute_end_cycle = self.cycle + latency

    def issue_instruction(self, instruction):
        if len(self.reorder_buffer) < self.buffer_sizes.get("reorder", 5):
            self.reorder_buffer.append(instruction)
            instruction.issue_cycle = self.cycle

    def commit_instruction(self, instruction):
        self.reorder_buffer.remove(instruction)
        instruction.commit_cycle = self.cycle

    def process_instructions(self):
        for instruction in self.instructions:
            self.issue_instruction(instruction)

            if instruction.opcode in ['flw', 'lw', 'fsw', 'sw']:
                while len(self.load_buffer) > 0 and self.load_buffer[0].commit_cycle:
                    self.load_buffer.pop(0)
            else:
                while len(self.reorder_buffer) > 0 and self.reorder_buffer[0].commit_cycle:
                    self.commit_instruction(self.reorder_buffer[0])

            self.execute_instruction(instruction)

            # Set read_result_cycle only when the instruction is executed
            if instruction.execute_end_cycle:
                instruction.read_result_cycle = instruction.execute_end_cycle + 1

            self.commit_instruction(instruction)
            self.cycle = instruction.read_result_cycle if instruction.read_result_cycle else self.cycle

    def print_simulation_results(self):
        print("Pipeline Simulation")
        print("-" * 60)
        print("{:<20} {:<7} {:<9} {:<7} {:<7} {:<7}".format(
            "Instruction", "Issues", "Executes", "Read", "Result", "Commits"
        ))
        print("{:<20} {:<7} {:<9} {:<7} {:<7} {:<7}".format(
            "---------------------", "------", "---------", "------", "------", "-------"
        ))
        for instruction in self.instructions:
            if instruction.execute_start_cycle:
                exec_range = f"{instruction.execute_start_cycle}-{instruction.execute_end_cycle}"
            else:
                exec_range = "-"

            if instruction.commit_cycle:
                commit_cycle = str(instruction.commit_cycle)
            else:
                commit_cycle = "-"

            if instruction.read_result_cycle:
                read_cycle = str(instruction.read_result_cycle)
            else:
                read_cycle = "-"

            print("{:<20} {:<7} {:<9} {:<7} {:<7} {:<7}".format(
                f"{instruction.opcode} {instruction.operands}", instruction.issue_cycle,
                exec_range, read_cycle, commit_cycle, instruction.commit_cycle
            ))

    def print_delays(self):
        print("\nDelays")
        print("-" * 30)
        print(f"reorder buffer delays: {sum(1 for instr in self.instructions if instr.issue_cycle and not instr.commit_cycle)}")
        print(f"reservation station delays: {sum(1 for instr in self.instructions if instr.issue_cycle and not instr.execute_start_cycle)}")
        print(f"data memory conflict delays: {sum(1 for instr in self.instructions if instr.opcode in ['flw', 'lw', 'fsw', 'sw'] and instr.execute_start_cycle and not instr.read_result_cycle)}")
        print(f"true dependence delays: {sum(1 for instr in self.instructions if instr.opcode not in ['flw', 'lw', 'fsw', 'sw'] and instr.execute_start_cycle and not instr.read_result_cycle)}")


# Example usage:
if __name__ == "__main__":
    config_file = "config.txt"
    trace_file = "trace.dat"

    processor = Processor(config_file)
    processor.read_trace(trace_file)
    processor.process_instructions()

    print("Configuration")
    print("-------------")
    # Read and print the pipeline configuration from config_file
    with open(config_file, 'r') as file:
        config = file.read()
        print(config)

    processor.print_simulation_results()
    processor.print_delays()


Configuration
-------------
buffers

eff addr: 2
fp adds: 3
fp muls: 3
ints: 2
reorder: 5

latencies

fp_add: 2
fp_sub: 2
fp_mul: 5
fp_div: 10


Pipeline Simulation
------------------------------------------------------------
Instruction          Issues  Executes  Read    Result  Commits
--------------------- ------  --------- ------  ------  -------
flw    f6,32(x2)     1       -         -       1       1      
flw    f2,48(x3)     1       -         -       1       1      
fmul.s f0,f2,f4      1       1-2       3       1       1      
fsub.s f8,f6,f2      3       3-4       5       3       3      
fdiv.s f10,f0,f6     5       5-6       7       5       5      
fadd.s f6,f8,f2      7       7-8       9       7       7      

Delays
------------------------------
reorder buffer delays: 0
reservation station delays: 2
data memory conflict delays: 0
true dependence delays: 0
